In [2]:
import requests
import json
from bs4 import BeautifulSoup 

In [4]:
CIKNumber = '0001341439'
filingType = "10-K"
baseURL = "https://www.sec.gov"

filingParameters = {'action':'getcompany',
                                  'CIK':CIKNumber,
                                  'type':filingType,
                                  'dateb':'',
                                  'owner':'exclude',
                                  'start':'',
                                  'output':'',
                                  'count':'100'}

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

apiResponse = requests.get(f"https://data.sec.gov/api/xbrl/companyfacts/CIK{CIKNumber}.json", headers=header)

if apiResponse.ok:
    apiData = json.loads(apiResponse.content)
else:
    print(apiResponse.status_code)
    print(apiResponse.reason)
    
# request the url, and then parse the response.
companyResponse = requests.get(url=r"https://www.sec.gov/cgi-bin/browse-edgar", params=filingParameters, headers=header)

#print(response.content)
companyParser = BeautifulSoup(companyResponse.content, 'html.parser')
documentLink = companyParser.find("a", {"id":"documentsbutton"})
print(baseURL + documentLink["href"])

documentResponse = requests.get(baseURL + documentLink["href"], headers=header)
#print(documentResponse.content)

documentParser = BeautifulSoup(documentResponse.content, 'html.parser')
xmlTable = documentParser.find("table", {"summary":"Data Files"})
xmlTableData = xmlTable.find_all("td", {"scope":"row"})

xmlSchemaData = None

i=0

while i < len(xmlTableData):
    if xmlTableData[i].getText() == "XBRL TAXONOMY EXTENSION CALCULATION LINKBASE DOCUMENT":
        xmlSchemaData = xmlTableData[i+1].find("a")
        break
    
    i+=1
    
print(baseURL + xmlSchemaData["href"])

schemaReportResponse = requests.get(baseURL + xmlSchemaData["href"], headers=header)
#print(schemaReportResponse.content)

schemaReportParser = BeautifulSoup(schemaReportResponse.content, features="xml")
financialStatements = schemaReportParser.find_all("link:calculationLink", {"xlink:type":"extended"})

data = {"statement": []}

for statement in financialStatements:
    statementName = statement["xlink:role"].split("role/")[1]
    
    try:
        statementName = statementName.split("Role_")[1]
    except:
        continue

    statementDictionary = {"name":statementName, "concept": []}
    
    
    concepts = statement.find_all("link:calculationArc", {"xlink:type":"arc"})
    
    for concept in concepts:
        try:
            conceptDictionary = {"section":concept["xlink:from"].split("us-gaap_")[1], "name":concept["xlink:to"].split("us-gaap_")[1], "data": apiData["facts"]["us-gaap"][concept["xlink:to"].split("us-gaap_")[1]]}
        except:
            try:
                conceptDictionary = {"section":concept["xlink:from"], "name":concept["xlink:to"], "data":apiData["facts"]["us-gaap"][concept["xlink:to"].split("us-gaap_")[1]]}
            except:
                continue
        finally:
            statementDictionary["concept"].append(conceptDictionary)
    
    data["statement"].append(statementDictionary)

print(data)

https://www.sec.gov/Archives/edgar/data/1341439/000095017023028914/0000950170-23-028914-index.htm
https://www.sec.gov/Archives/edgar/data/1341439/000095017023028914/orcl-20230531_cal.xml
{'statement': [{'name': 'StatementCONSOLIDATEDBALANCESHEETS', 'concept': [{'section': 'LiabilitiesAndStockholdersEquity', 'name': 'StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest', 'data': {'label': "Stockholders' Equity, Including Portion Attributable to Noncontrolling Interest", 'description': "Amount of stockholders' equity (deficit), net of receivables from officers, directors, owners, and affiliates of the entity, attributable to both the parent and noncontrolling interests. Amount excludes temporary equity. Alternate caption for the concept is permanent equity.", 'units': {'USD': [{'end': '2007-05-31', 'val': 17235000000, 'accn': '0001193125-10-151896', 'fy': 2010, 'fp': 'FY', 'form': '10-K', 'filed': '2010-07-01', 'frame': 'CY2007Q2I'}, {'end': '2008-05-31', 'val': 2339400